In [2]:
import pandas as pd
import numpy as np
import os
from ultrack.tracks.graph import get_paths_to_roots, tracks_df_forest

In [3]:
# load tracks dataset
# root = "/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/killi_tracker/"

root = "E:/Dropbox (Cole Trapnell Lab)/Nick Lammers/Nick/killi_tracker/"

project_name = "20250311_LCP1-NLSMSCl"
tracking_config = "tracking_20250328_redux"
tracking_name = "track_0000_2339_cb"

track_path = os.path.join(root, "tracking", project_name, tracking_config, "well0000", tracking_name, "tracks_fluo.csv")
tracks_df = pd.read_csv(track_path)

FileNotFoundError: [Errno 2] No such file or directory: 'E:/Dropbox (Cole Trapnell Lab)/Nick Lammers/Nick/killi_tracker/tracking\\20250311_LCP1-NLSMSCl\\tracking_20250328_redux\\well0000\\track_0000_2339_cb\\tracks_fluo.csv'

In [ ]:
import plotly.express as px

tid, tc = np.unique(tracks_df["track_id"], return_counts=True)
counts_df = pd.DataFrame(tid, columns=["track_id"])
counts_df["track_length"] = tc
tracks_df = tracks_df.merge(counts_df, how="left", on="track_id")

In [ ]:
fluo_thresh = 115

bright_tracks = tracks_df.loc[tracks_df["mean_fluo"] > fluo_thresh, "track_id"]

ti, tc = np.unique(bright_tracks, return_counts=True)
candidate_tracks = ti[tc>0]

tracks_df["track_id_str"] = tracks_df["track_id"].astype(str)

# fig = px.scatter(tracks_df.loc[np.isin(tracks_df["track_id"], candidate_tracks)], x="t", y="mean_fluo", color="track_id_str")
fig = px.scatter(tracks_df.loc[tracks_df["mean_fluo"]>fluo_thresh], x="t", y="mean_fluo", color="track_id_str")
fig.show()

In [ ]:
fi, fc = np.unique(tracks_df.loc[tracks_df["mean_fluo"]>fluo_thresh, "t"], return_counts=True)

fig = px.scatter(x=fi, y=fc)
fig.show()

### Segments look shite. What about the raw masks?
Manual inspection indicates that a number of raw masks corresponding to lcp+ nuclei are beingd dropped durring tracking, which is frustrating

In [ ]:
from glob2 import glob
from tqdm import tqdm

fluo_path = os.path.join(root, "built_data", "fluorescence_data", project_name, "")
fluo_df_path_list = sorted(glob(fluo_path + "*.csv"))
fluo_df_list = []
for fluo_p in tqdm(fluo_df_path_list):
    df = pd.read_csv(fluo_p)
    fluo_df_list.append(df)

fluo_df = pd.concat(fluo_df_list, axis=0, ignore_index=True)

In [ ]:
print(np.sum(fluo_df["mean_fluo"]>fluo_thresh))
print(np.sum(tracks_df["mean_fluo"]>fluo_thresh))

We see substantially more high-fluo frames. Let's look at trends over time

In [ ]:
N = 50  # for example

# Group by time 't' and, for each group, pick the N rows with the highest 'mean_fluo'
top_fluo_df = fluo_df.groupby('frame', group_keys=False).apply(lambda x: x.nlargest(N, columns='mean_fluo')).reset_index(drop=True)

In [ ]:
fig = px.scatter(top_fluo_df, x="frame", y="mean_fluo")
fig.show()

In [ ]:
fi, fc = np.unique(fluo_df.loc[fluo_df["mean_fluo"]>fluo_thresh, "frame"], return_counts=True)

fig = px.scatter(x=fi, y=fc)
fig.show()

Clearly we're losing a ton during the tracking process. Sad.

### Assess overall quality of the tracks. Can we reconstruct lineage trees?

In [ ]:
from ultrack.tracks.graph import inv_tracks_df_forest

forest_graph = tracks_df_forest(tracks_df)
inv_forest_graph = inv_tracks_df_forest(tracks_df)

In [ ]:
def get_root(cell, parent_map):
    """
    Recursively follow the parent mapping until a cell is reached that has no parent.
    Assumes parent_map[cell] returns a list of parent IDs (with one parent per cell).
    """
    while cell in parent_map:
        # For a simple 1-to-1 mapping, take the first (and only) parent.
        cell = parent_map[cell]
    return cell

# Build a list of results for each child that is a key in parent_map.
results = []
track_index = np.unique(tracks_df["track_id"])
mapped_ids = np.asarray(list(inv_forest_graph.keys()))
for child in tqdm(track_index):
    if child in mapped_ids:
        root = get_root(child, inv_forest_graph)
    else:
        root = child
    # Look up the frame number for the root cell
    root_frame = np.min(tracks_df.loc[tracks_df["track_id"]==root, "t"])
    leaf_frame = np.max(tracks_df.loc[tracks_df["track_id"]==child, "t"])
    results.append({'child_id': child, 'root_id': root, 'root_frame': root_frame, 'leaf_frame': leaf_frame})

# Convert results to a DataFrame
df_roots = pd.DataFrame(results)
df_roots = df_roots.merge(counts_df, how="left", left_on="child_id", right_on="track_id")

In [ ]:
df_roots_ft = df_roots.loc[df_roots["track_length"] >= 10]
print(df_roots_ft.shape)

In [ ]:
df_roots_ft["span"] = df_roots_ft["leaf_frame"] - df_roots_ft["root_frame"]

fig = px.scatter(df_roots_ft, x="leaf_frame", y="span")
fig.show()

In [ ]:
from ultrack.tracks.gap_closing import close_tracks_gaps

test = close_tracks_gaps(tracks_df, max_gap=3, max_radius=50, scale=np.asarray([3.0, 1.0, 1.0]))

In [ ]:
test

In [ ]:
len(np.unique(test["track_id"]))

In [ ]:
len(np.unique(tracks_df["track_id"]))